In [8]:
import sys, os
sys.path.insert(0, '../..')

from project_utils import *
from matrices import *
from functional_connectivity import * 
from regression import * 
import second_level 

# analysis specific
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV

from nilearn import plotting

import warnings
warnings.filterwarnings("ignore")
from matplotlib.gridspec import GridSpec

def get_lt_pairs(singles):
    pairs = []
    for i1 in range(0, len(singles) + 1):
        for i2 in range(i1 + 1, len(singles)):
            pairs.append([singles[i1], singles[i2]])
    return pairs

# directories
syn_dir  = '/Volumes/synapse/projects/SocialSpace/Projects/SNT-fmri_CUD'
lsa_dir  = syn_dir + '/Analyses/GLMs/LSA/fmriprep-rp+fd+acc_csf'
fc_dir   = lsa_dir + '/connectivity_images'
beh_dir  = syn_dir + '/Data/Behavior/Pmod_analyses'
mask_dir = syn_dir + '/Masks'
pmod_fnames = glob.glob(beh_dir + '/*pmods*')

In [ ]:
coords = {'LIFG': [-45, 41, -2],
          'RHPC_ant': [24, -36, 1], 'RHPC_mid': [24, -20, -13], 'RHPC_post':[24, -3, -27]}

In [102]:
sub_info = pd.read_excel('../../../Info/participants_info.xlsx')
sub_info[(sub_info['dx']=='CD') & (sub_info['incl']==1)]['sub_id'].values
sub_info = sub_info[sub_info['incl'] == 1]
sub_info.sort_values(by='sub_id', inplace=True)
sub_info = sub_info.merge(df, on=['sub_id','dx'])

# Load fMRI data

In [ ]:
conn_img_dir = lsa_dir + '/func_conn/connectivity_images'
conn_imgs = glob.glob(conn_img_dir + '/sub*')

coords = [(-22,-15,-18)]
fig, axs = plt.subplots(figsize=(10, 8), nrows=5)
for c, conn_img in enumerate(conn_imgs[1:6]):
    
    display = plotting.plot_stat_map(conn_img, 
                                    threshold=0.20, vmax=1,
                                    cut_coords=coords[0],
                                    title='', figure=fig, axes=axs[c])
    display.add_markers(marker_coords=coords, marker_color='black', marker_size=100)
    plt.show()

# CanICA
https://nilearn.github.io/stable/auto_examples/03_connectivity/plot_compare_decomposition.html#sphx-glr-auto-examples-03-connectivity-plot-compare-decomposition-py

In [ ]:
canica = CanICA(n_components=20,
                memory="nilearn_cache", memory_level=2,
                verbose=10,
                mask_strategy='whole-brain-template',
                random_state=0)
canica.fit(beta_img_fnames[0:2])
# canica_components_img.to_filename('canica.nii.gz')
for r in range(5):
    display = plotting.plot_stat_map(canica.components_img_.slicer[:,:,:,r], title='')

# Correlate w/ self-report variables

In [ ]:
[c for c in df.columns if 'mri' in c]

#### Filter for possible interesting correlations
- focus on regions previously found: e.g., left hippocampus, precuneus/pcc

In [ ]:
sr_corr_df = pd.DataFrame(columns=['n', 'fc_region1', 'fc_region2', 'self-report', 'pvalue', 'beta'])
for sr_col in ['ctq_total_score_2', 
               'sni_num_ppl', 'cssa_total_mri',
               'lsas_social_interaction_fear_subscale', 
               'coc_age_1st_use', 'coc_days_last_use', 'mri_utox___coc']:
        
    X_labels = [sr_col]
                
    if 'coc' not in sr_col: # control for diagnosis if it's not already a cocaine variable
        X_labels.append('C(dx)')
        X_labels.append('C(dx)*' + sr_col)
        mask = np.isfinite(corr_df_z[sr_col])
    else: # ensure its only CUDs
        mask = np.isfinite(corr_df_z[sr_col]) & (corr_df_z['dx'] == 'CD')
    n = np.sum(mask)
                   
    for fc_col in label_cols:
        
        # should also filter for the other region too
        if 'Hippocampus' in fc_col or 'Precun' in fc_col or 'Cingulate' in fc_col: # simplify
            
            # to speed up, filter on bivariate correlation
            _, cor_p = scipy.stats.pearsonr(corr_df_z[fc_col][mask], corr_df_z[sr_col][mask])
            if cor_p < 0.05: 

                # add any interactions
                res_df, ols = run_ols(X_labels, fc_col, corr_df_z[mask], covariates=['age_years', 'sex', 'asi_education'])
                ols_p       = res_df[sr_col + '_pvalue'].values[0]
                ols_b       = res_df[sr_col + '_beta'].values[0]
                if ols_p < 0.005:
                    sr_corr_df.loc[len(sr_corr_df) + 1, :] = [n, fc_col.split('_')[0], fc_col.split('_')[1], sr_col, ols_p, ols_b]
sr_corr_df